In [11]:
import numpy as np
def get_condition_indices(stabilizer_shape):
    """
    Extracts the list of relative positions (dx, dy) from a 3x3 binary rule matrix
    that define which neighbor cells contribute to the update rule.

    The (0, *) row is ignored (assumed to represent the current cell),
    and only rows 1 and 2 are used to define condition offsets.
    """
    rows, cols = stabilizer_shape.shape
    relative_positions = [[(0, 0), (0, 0), (0, 0)], # row 0 (ignored because it's result of input)
                   [(1, -1), (1, 0), (1, 1)], # row 1
                   [(2, -1), (2, 0), (2, 1)]] # row 2

    active_conditions = []               

    for i in range(1, rows):
        for j in range(cols):
            if stabilizer_shape[i][j] == 1:
                active_conditions.append(relative_positions[i][j])

    return active_conditions

def fill_Z_with_stabilizer_shape(input_row, H, L, m, condition_offsets_list):
    """Evolve the automaton from the input_row using given rule offsets."""
    Z = np.zeros((H - m + 1, L), dtype=int)
    Z = np.append(Z, input_row, axis=0)  # append input row at the bottom
    for i in range(H - m, -1, -1):  # evolve upward
        condition_offsets = condition_offsets_list[H - m - i]  # get the condition offsets for this row
        for j in range(L):
            neighbor_sum = 0
            for dx, dy in condition_offsets:
                neighbor_sum += Z[i + dx, (j + dy) % L]
            Z[i, j] = 1 if neighbor_sum % 2 == 1 else 0  # parity check

    return Z

def precompute_basis_outputs(H, L, m, condition_indices):
    """Precompute Z for each basis input (single 1 at position i)."""
    Z_basis = []

    for i in range(m-1):
        input_i = np.zeros((1, (m-1) * L), dtype=int)
        input_i = input_i.reshape(m - 1, L)
        input_i[i, 0] = 1
        Z_first = fill_Z_with_stabilizer_shape(input_i, H, L, m, condition_indices)
        Z_basis.append(Z_first)

        for j in range(1, L):
            Z_j = np.roll(Z_first, j, axis=1)
            Z_basis.append(Z_j)

    """Print the precomputed Z_basis for debugging"""
    # print(f"Precomputed Z_basis for H={H}, L={L}, m={m}:")
    # for idx, z in enumerate(Z_basis):
    #     print(f"Z_basis[{idx}] (index {idx}):\n{z}")

    return Z_basis

def evolve_via_basis(input_row, Z_basis, L, min_weight):
    """Use linearity to compute Z for arbitrary input row."""
    result = np.zeros_like(Z_basis[0])
    indices = np.transpose(np.nonzero(input_row))
    for x, y in indices:
        idx = x * L + y  # compute index in Z_basis
        result = np.bitwise_xor(result, Z_basis[idx])
    return result

from itertools import product

def find_distance_via_basis(Z_basis, H, L, m, pattern_inputs=None):
    """Find the minimum weight Z by iterating over all possible input rows."""
    
    n = H * L
    k = (m - 1) * L
    min_weight = H * L  # initialize large
    kd_over_n = 0

    if pattern_inputs is None:
        iterator = product([0, 1], repeat=k)
    else:
        iterator = pattern_inputs


    for item in iterator:
        if pattern_inputs is None:
            bits = item
        else:
            bits = list(map(int, format(item, f'0{k}b')))

        input_row = np.array(bits, dtype=int).reshape(m-1, L)
        Z = evolve_via_basis(input_row, Z_basis, L, min_weight)
        weight = np.sum(Z)
        if weight > 0:
            if weight < min_weight:
                # New lower weight found: reset lists
                min_weight = weight
                kd_over_n = (k * min_weight) / n
                min_weight_inputs = [input_row.copy()]
                min_weight_outputs = [Z.copy()]
                print(f"Found Z with d={weight}: \n{Z}")
            elif weight == min_weight:
                # Same as current min_weight: add to list
                min_weight_inputs.append(input_row.copy())
                min_weight_outputs.append(Z.copy())
                print(f"Found Z with d={weight}: \n{Z}")
    print(f"best kd/n = {kd_over_n:.2f}")
    return min_weight_inputs, min_weight_outputs, k, min_weight, n, kd_over_n



In [12]:
from itertools import combinations

def rotate_left(x, k, bits):
    """Rotate k-bit integer x left by 1."""
    return ((x << 1) & ((1 << bits) - 1)) | (x >> (bits - 1))

def generate_unique_cyclic_inputs(k):
    seen = set()
    unique = []

    for weight in range(1, k + 1):
        for positions in combinations(range(k), weight):
            # Create integer representation
            val = sum(1 << (k - 1 - pos) for pos in positions)

            # Generate all cyclic rotations
            rotations = set()
            x = val
            for _ in range(k):
                rotations.add(x)
                x = rotate_left(x, 1, k)

            # Skip if any rotation is already seen
            if not seen.isdisjoint(rotations):
                continue

            # Mark all rotations as seen
            seen.update(rotations)
            unique.append(val)

    return unique

# # Test for m=3, L=4 -> k = 8
# m = 3
# L = 2
# k = (m - 1) * L

# unique_pattern_inputs = generate_unique_cyclic_inputs(k)
# print(f"Unique inputs under cyclic symmetry (k={k}): {len(unique_pattern_inputs)}")
# for val in sorted(unique_pattern_inputs):
#     print(f"{val:0{k}b}")  # Print as zero-padded binary strings


In [13]:
H, L = 4, 5
m = 3
stabilizer_shapes = [
    np.array([[0, 1, 0],
              [1, 0, 0],
              [1, 1, 0]]),
    np.array([[0, 1, 0],
              [1, 1, 0],
              [0, 0, 1]])
]

print("Stabilizer shapes:")
print(stabilizer_shapes)
condition_indices_list = []

for shape in stabilizer_shapes:
    assert shape.shape == (3, 3), "Stabilizer shape must be 3x3."
    condition_indices_list.append(get_condition_indices(shape))

print(f"Condition indices extracted: {condition_indices_list}")

Z_basis = precompute_basis_outputs(H, L, m, condition_indices_list)

unique_pattern_inputs = generate_unique_cyclic_inputs((m - 1) * L)

# Find minimum weight Z with Brute Force search
# min_weight_inputs, min_weight_outputs, k, d, n, kd_over_n = find_distance_via_basis(Z_basis, H, L, m)
# print(f"k: {k}, d: {d}, n: {n}, kd/n: {kd_over_n:.2f}")
# Find minimum weight Z with Brute Force search using unique cyclic inputs
print("\nFinding minimum weight Z using unique cyclic inputs...")
print(f"Using unique cyclic inputs of length {k}: {len(unique_pattern_inputs)}")
print("Unique cyclic inputs:")
for val in sorted(unique_pattern_inputs):
    print(f"{val:0{k}b}")  # Print as zero-padded binary strings

min_weight_inputs, min_weight_outputs, k, d, n, kd_over_n = find_distance_via_basis(
    Z_basis, H, L, m, pattern_inputs=unique_pattern_inputs
)
print(f"k: {k}, d: {d}, n: {n}, kd/n: {kd_over_n:.2f}")

Stabilizer shapes:
[array([[0, 1, 0],
       [1, 0, 0],
       [1, 1, 0]]), array([[0, 1, 0],
       [1, 1, 0],
       [0, 0, 1]])]
Condition indices extracted: [[(1, -1), (2, -1), (2, 0)], [(1, -1), (1, 0), (2, 1)]]

Finding minimum weight Z using unique cyclic inputs...
Using unique cyclic inputs of length 10: 107
Unique cyclic inputs:
1000000000
1000010000
1000100000
1001000000
1001001000
1010000000
1010000100
1010001000
1010010000
1010010100
1010100000
1010100100
1010101000
1010101010
1100000000
1100000010
1100000100
1100001000
1100001010
1100010000
1100010010
1100010100
1100011000
1100100000
1100100010
1100100100
1100101000
1100101010
1100110000
1100110010
1101000000
1101000010
1101000100
1101001000
1101001010
1101001100
1101010000
1101010010
1101010100
1101011000
1101011010
1101100000
1101100010
1101100100
1101101000
1101101010
1101101100
1110000000
1110000010
1110000100
1110000110
1110001000
1110001010
1110001100
1110010000
1110010010
1110010100
1110010110
1110011000
1110011010
